In [ ]:
import cv2


# 读入图像
img = cv2.imread('1.jpg')

# 转换为灰度图像
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 使用Otsu's方法进行二值化
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)


# 定义一个5x5的结构元素
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

# 进行开运算
opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)


In [ ]:
import cv2
import pytesseract
from pytesseract import Output

# 读入图像
img = cv2.imread('1.jpg')

# 使用 Tesseract 进行文本检测，返回结果的数据结构为字典
d = pytesseract.image_to_data(img, output_type=Output.DICT)

# 遍历所有检测到的文本区域
n_boxes = len(d['text'])
for i in range(n_boxes):
    # 如果文本区域的置信度不为0
    if int(d['conf'][i]) > 0:
        # 获取文本区域的坐标
        (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])

        # 在原图上画出文本区域
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# 显示带有文本区域的图像
cv2.imshow('img', img)
cv2.waitKey(0)


In [ ]:
!git clone https://github.com/clovaai/CRAFT-pytorch.git


In [ ]:
%cd CRAFT-pytorch


In [ ]:
%pip install -r requirements.txt

In [ ]:
!python test.py --trained_model=[path_to_your_weight_file] --test_folder=[path_to_your_images]


In [2]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('1.jpg')

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Use Otsu's method to binarize the image
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Threshold for empty space
vertical_empty_space_threshold = binary.shape[1] * 255 * 0.02  # Adjust this value according to your image
horizontal_empty_space_threshold = binary.shape[0] * 255 * 0.02  # Adjust this value according to your image

# Initialize the previous split point
prev_vertical_split_point = 0

# Compute the vertical projection of the binary image
vertical_projection = np.sum(binary, axis=1)

# Find the empty spaces
vertical_empty_spaces = np.where(vertical_projection <= vertical_empty_space_threshold)[0]

# Initialize the list of split points
vertical_split_points = []

# Initialize the previous empty space
prev_vertical_empty_space = -1

# Loop over the empty spaces
for empty_space in vertical_empty_spaces:
    if prev_vertical_empty_space != -1 and empty_space - prev_vertical_empty_space > 1:
        # If there is a gap between two empty spaces, add a split point
        vertical_split_points.append((prev_vertical_empty_space + empty_space) // 2)
    prev_vertical_empty_space = empty_space

# Add the end of the image as the last split point
vertical_split_points.append(binary.shape[0])

# Loop over the split points
for i, vertical_split_point in enumerate(vertical_split_points):
    # Cut the image at the split point
    vertical_part = img[prev_vertical_split_point:vertical_split_point, :]

    # Compute the horizontal projection of the binary image
    horizontal_projection = np.sum(binary[prev_vertical_split_point:vertical_split_point, :], axis=0)

    # Find the empty spaces
    horizontal_empty_spaces = np.where(horizontal_projection <= horizontal_empty_space_threshold)[0]

    # Initialize the list of split points
    horizontal_split_points = []

    # Initialize the previous empty space
    prev_horizontal_empty_space = -1

    # Loop over the empty spaces
    for empty_space in horizontal_empty_spaces:
        if prev_horizontal_empty_space != -1 and empty_space - prev_horizontal_empty_space > 1:
            # If there is a gap between two empty spaces, add a split point
            horizontal_split_points.append((prev_horizontal_empty_space + empty_space) // 2)
        prev_horizontal_empty_space = empty_space

    # Add the end of the image as the last split point
    horizontal_split_points.append(vertical_part.shape[1])

    # Initialize the previous split point
    prev_horizontal_split_point = 0

    # Loop over the split points
    for j, horizontal_split_point in enumerate(horizontal_split_points):
        # Cut the image at the split point
        part = vertical_part[:, prev_horizontal_split_point:horizontal_split_point]

        # Save the part
        cv2.imwrite(f'picture/part_{i}_{j}.jpg', part)

        # Update the previous split point
        prev_horizontal_split_point = horizontal_split_point

    # Update the previous split point
    prev_vertical_split_point = vertical_split_point


In [3]:
import cv2
import numpy as np

img = cv2.imread('1.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

sobelx = cv2.Sobel(binary, cv2.CV_64F, 1, 0, ksize=3)
gradient = cv2.convertScaleAbs(sobelx)

edge_region_threshold = 50

vertical_projection = np.sum(gradient, axis=1)

edge_regions = np.where(vertical_projection >= edge_region_threshold)[0]

vertical_split_points = []

prev_edge_region = -1

for edge_region in edge_regions:
    if prev_edge_region != -1 and edge_region - prev_edge_region > 1:
        vertical_split_points.append((prev_edge_region + edge_region) // 2)
    prev_edge_region = edge_region

vertical_split_points.append(binary.shape[0])

prev_vertical_split_point = 0

for i, vertical_split_point in enumerate(vertical_split_points):
  
    vertical_part = img[prev_vertical_split_point:vertical_split_point, :]

    gray_part = cv2.cvtColor(vertical_part, cv2.COLOR_BGR2GRAY)

    _, binary_part = cv2.threshold(gray_part, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    horizontal_projection = np.sum(binary_part, axis=0)

    horizontal_empty_spaces = np.where(horizontal_projection <= horizontal_empty_space_threshold)[0]

    horizontal_split_points = []

    prev_horizontal_empty_space = -1

    for empty_space in horizontal_empty_spaces:
        if prev_horizontal_empty_space != -1 and empty_space - prev_horizontal_empty_space > 1:
            horizontal_split_points.append((prev_horizontal_empty_space + empty_space) // 2)
        prev_horizontal_empty_space = empty_space

    horizontal_split_points.append(vertical_part.shape[1])

    prev_horizontal_split_point = 0

    for j, horizontal_split_point in enumerate(horizontal_split_points):
        part = vertical_part[:, prev_horizontal_split_point:horizontal_split_point]

        cv2.imwrite(f'picture/part_{i}_{j}.jpg', part)

        prev_horizontal_split_point = horizontal_split_point

    prev_vertical_split_point = vertical_split_point


In [ ]:
import paddlehub as hub
import cv2

# 加载移动端预训练模型
ocr = hub.Module(name="chinese_ocr_db_crnn_mobile")

# 服务端可以加载大模型，效果更好 -- 【个人电脑，内存不够用】
# ocr = hub.Module(name="chinese_ocr_db_crnn_server")

# 将预测图片存放在一个文件中(picture.txt)
test_img_path = []
with open('picture.txt', 'r') as f:
    for line in f:
        test_img_path.append(line.strip())
print("预测图片 => ", test_img_path)

# 读取测试文件夹test.txt中的照片路径
np_images = [cv2.imread(image_path) for image_path in test_img_path]

results = ocr.recognize_text(
    images=np_images,  # 图片数据，ndarray.shape 为 [H, W, C]，BGR格式；
    use_gpu=False,  # 是否使用 GPU；若使用GPU，请先设置CUDA_VISIBLE_DEVICES环境变量
    output_dir='ocr_result',  # 图片的保存路径，默认设为 ocr_result；
    visualization=True,  # 是否将识别结果保存为图片文件；
    box_thresh=0.5,  # 检测文本框置信度的阈值；
    text_thresh=0.5)  # 识别中文文本置信度的阈值；

for result in results:
    data = result['data']
    save_path = result['save_path']
    for infomation in data:
        print('text: ', infomation['text'], '\nconfidence: ', infomation['confidence'], '\ntext_box_position: ', infomation['text_box_position'])



In [7]:
import paddlehub as hub
import cv2

# 加载移动端预训练模型
ocr = hub.Module(name="chinese_ocr_db_crnn_mobile")

# 预测图片
test_img_path = ["1.jpg"]
print("预测图片 => ", test_img_path)

# 读取测试文件夹test.txt中的照片路径
np_images = [cv2.imread(image_path) for image_path in test_img_path]

results = ocr.recognize_text(
    images=np_images,  # 图片数据，ndarray.shape 为 [H, W, C]，BGR格式；
    use_gpu=False,  # 是否使用 GPU；若使用GPU，请先设置CUDA_VISIBLE_DEVICES环境变量
    output_dir='ocr_result',  # 图片的保存路径，默认设为 ocr_result；
    visualization=True,  # 是否将识别结果保存为图片文件；
    box_thresh=0.5,  # 检测文本框置信度的阈值；
    text_thresh=0.5)  # 识别中文文本置信度的阈值；

for result in results:
    data = result['data']
    save_path = result['save_path']
    for infomation in data:
        print('text: ', infomation['text'], '\nconfidence: ', infomation['confidence'], '\ntext_box_position: ', infomation['text_box_position'])


预测图片 =>  ['1.jpg']
Download https://bj.bcebos.com/paddlehub/paddlehub_dev/chinese_text_detection_db_mobile_1.1.0.zip
[##################################################] 100.00%
Decompress C:\Users\NIU\.paddlehub\tmp\tmp1vt9bmkz\chinese_text_detection_db_mobile_1.1.0.zip
[##################################################] 100.00%


[2023-06-10 08:53:12,554] [    INFO] - Successfully installed dependent packages.U\.paddlehub\tmp\tmp6ui35gxs\chinese_text_detection_db_mobile\requirements.txt: /
[2023-06-10 08:53:12,799] [    INFO] - Successfully installed chinese_text_detection_db_mobile-1.1.0


text:  自我评价 
confidence:  0.989576518535614 
text_box_position:  [[627, 38], [744, 38], [744, 69], [627, 69]]
text:  本人性格开朗、稳重、有活力，待人热情、真诚。有较强的组织 
confidence:  0.9970336556434631 
text_box_position:  [[575, 85], [1197, 87], [1197, 113], [575, 111]]
text:  能力、团体协作精神，较好的社交能力，善于处理各种人际关系。 
confidence:  0.9829587936401367 
text_box_position:  [[579, 129], [1193, 129], [1193, 153], [579, 153]]
text:  能迅速的适应各种环境，并融合其中。能把企业当作家庭，企业 
confidence:  0.9984939694404602 
text_box_position:  [[577, 171], [1197, 171], [1197, 195], [577, 195]]
text:  的财富就是我的财富，在努力为企业服务的过程中实现自身价值 
confidence:  0.9995457530021667 
text_box_position:  [[575, 213], [1193, 213], [1193, 237], [575, 237]]
text:  13899999999 
confidence:  0.9987695813179016 
text_box_position:  [[968, 308], [1127, 308], [1127, 334], [968, 334]]
text:  张吉椎 
confidence:  0.8467123508453369 
text_box_position:  [[172, 327], [311, 327], [311, 372], [172, 372]]
text:  求职意向：行政管理类 
confidence:  0.9985655546188354 
text_box_position:  [[445, 336], [697

In [10]:
import os
import cv2
import paddlehub as hub

# 加载移动端预训练模型
ocr = hub.Module(name="chinese_ocr_db_crnn_mobile")

# 读取文件夹中的所有图片
folder_path = 'picture'
test_img_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path) if img.endswith(('.png', '.jpg', '.jpeg'))]
print("预测图片 => ", test_img_paths)

# 读取测试文件夹中的照片路径
np_images = [cv2.imread(image_path) for image_path in test_img_paths]

results = ocr.recognize_text(
    images=np_images,  
    use_gpu=False,  
    output_dir='ocr_result',  
    visualization=True,  
    box_thresh=0.5,  
    text_thresh=0.5)

# 新建文件用于存储所有识别的文字
with open('output.txt', 'w') as f:
    for result in results:
        data = result['data']
        for information in data:
            f.write('text: ' + information['text'] + '\n')

预测图片 =>  ['picture\\part_0_0.png', 'picture\\part_0_1.png', 'picture\\part_1_0.png', 'picture\\part_2_0.png', 'picture\\part_3_0.png', 'picture\\part_4_0.png']
